In [101]:
import pandas as pd
from googletrans import Translator
from medscraper.tools import tidy


In [102]:
url="https://www.moh.gov.gr/articles/times-farmakwn/deltia-timwn/11488-epikairopoihsh-deltioy-timwn-farmakwn-anthrwpinhs-xrhshs-me-enswmatwsh-dioikhtikwn-metabolwn?fdl=25415"

In [103]:
grecia=pd.read_excel(url,#Seleccionar columnas a leer
                        usecols=[2,3,7,8,10])
translator = Translator()
#Traducimos las columnas
grecia.columns = [translator.translate(col).text for col in grecia.columns]
grecia.columns=grecia.columns.str.lower()
#Renmbrar columnas
grecia=grecia.rename(columns={'active substance':'sustancia', 'retail price':'precioiva'})
#Concatenar product con active substance
grecia['busqueda']=grecia['sustancia']+' '+grecia['product']
#Limpia columna vat
grecia['vat']=grecia['vat'].str.replace('%','')
grecia['vat']=grecia['vat'].str.replace(' ','')
#Float
grecia['vat']=grecia['vat'].astype(float)
#Sustituir nan por 6
grecia['vat']=grecia['vat'].fillna(6)
#Precio sin iva
grecia['precio']=round(grecia['precioiva']*(1-grecia['vat']/100),2)
grecia

,product,atc,precioiva,sustancia,vat,busqueda,precio
0,NORBAL TAB 10MG/TAB BTx20 (BLIST 1x 20),N05BE01,4.55,BUSPIRONE,6.0,BUSPIRONE NORBAL TAB 10MG/TAB BTx20 (BLIST 1x 20),4.28
1,ZORTAL F.C.TAB 50MG/TAB ΒΤx14 (BLISTERS),N06AB06,4.17,SERTRALINE,6.0,SERTRALINE ZORTAL F.C.TAB 50MG/TAB ΒΤx14 (BLIS...,3.92
2,DORALIN F.C.TAB 40MG/TAB ΒΤx30 (BLIST 3x10),A03AB06,6.69,OTILONIUM,6.0,OTILONIUM DORALIN F.C.TAB 40MG/TAB ΒΤx30 (BLIS...,6.29
3,MAXUDIN TAB 20MG/TAB BTx14 (BLIST 1x14),C10AA03,4.07,PRAVASTATIN,6.0,PRAVASTATIN MAXUDIN TAB 20MG/TAB BTx14 (BLIST ...,3.83
4,MAXUDIN TAB 40MG/TAB BTx14 (BLIST1x14),C10AA03,7.02,PRAVASTATIN,6.0,PRAVASTATIN MAXUDIN TAB 40MG/TAB BTx14 (BLIST1...,6.60
...,...,...,...,...,...,...,...
8148,PRESSURIL TAB 20MG/TAB BTX30(BLIST 3X10),C09AA03,5.48,LISINOPRIL,6.0,LISINOPRIL PRESSURIL TAB 20MG/TAB BTX30(BLIST ...,5.15
8149,PRESSURIL TAB 40MG/TAB BTX30 (BLIST 3X10),C09AA03,9.46,LISINOPRIL,6.0,LISINOPRIL PRESSURIL TAB 40MG/TAB BTX30 (BLIST...,8.89
8150,REDUSTEROL F.C.TAB 20MG/TAB BTX30(3 BLIST X10),C10AA01,6.76,SIMVASTATIN,6.0,SIMVASTATIN REDUSTEROL F.C.TAB 20MG/TAB BTX30(...,6.35
8151,REDUSTEROL F.C.TAB 40MG/TAB BTX30(3BLIST X10),C10AA01,7.90,SIMVASTATIN,6.0,SIMVASTATIN REDUSTEROL F.C.TAB 40MG/TAB BTX30(...,7.43


In [118]:
def busqueda(busqueda):
    # Instantiate translator
    translator = Translator()
    
    # Traducción
    busqueda = translator.translate(busqueda, src='es', dest='en').text
    # Perform search in the DataFrame
    results = grecia[grecia['busqueda'].str.contains(busqueda, case=False)].copy()
    # Translate product names back to Spanish
    results['product'] = results['product'].apply(lambda x: translator.translate(x, src='el', dest='es').text)
    
    # Return the results
    return results[['product', 'precio']]


In [122]:
busqueda('pravastatina')

,product,precio
3,MAXUDIN TAB 20MG/TAB BTx14 (BLIST 1x14),3.83
4,MAXUDIN TAB 40MG/TAB BTx14 (BLIST1x14),6.60
750,PRAVALONG TAB 20MG/TAB BTx14 (BLIST 2x7),3.20
751,PRAVALONG TAB 20MG/TAB BTx28 (BLIST 4x7),5.28
752,PRAVALONG TAB 40MG/TAB BTx14 (BLIST 2x7),4.28
753,PRAVALONG TAB 40MG/TAB BTx28 (BLIST 4x7),8.58
1618,ASTO-CHOL TAB 40MG/TAB BTx30 (BLIST 3x10),8.87
2851,PRAVIN TAB 40MG/TAB BTX30(BLIST3X10),8.87
3335,PRAVAFENIX TAPONES (40+160)MG/TAP BOTELLA (HDP...,16.00
3390,PRAVALIP TAB 20MG/TAB BTx28 (BLIST 2x14),5.28
